In [16]:
import pandas as pd
import numpy as np

dfMovies = pd.read_csv('content/IMDb movies.csv', dtype={"year": str})
dfNames = pd.read_csv('content/IMDb names.csv')
dfRatings = pd.read_csv('content/IMDb ratings.csv')
dfTitlePrincipals = pd.read_csv('content/IMDb title_principals.csv')

# just run this cell once to load data into memory


In [17]:
dfMovies = dfMovies[['imdb_title_id', 'title', 'year','genre', 'duration', 'country', 'language', 'director', 'writer', 'production_company', 'budget', 'worlwide_gross_income']]
dfNames = dfNames[['imdb_name_id', 'name']]
dfRatings = dfRatings[['imdb_title_id', 'weighted_average_vote', 'total_votes', 'mean_vote' ]] # come back to this and include age groups/male/female votes
dfTitlePrincipals = dfTitlePrincipals[['imdb_title_id', 'ordering', 'imdb_name_id']] # shouyld we get category? "actress"/"actor" (dont know if its necessary)

labelVector = dfRatings['weighted_average_vote']




# dfMovies = pd.merge(dfMovies, dfRatings, on=["imdb_title_id"]) #@Jayden, this is sql equivalent inner join on imdb_title_id


In [18]:
dfMovies.year = dfMovies.year.str.extract('(\d+)')
dfMovies.budget = dfMovies.budget.str.extract('(\d+)')
dfMovies.worlwide_gross_income = dfMovies.worlwide_gross_income.str.extract('(\d+)')

dfMovies.year = pd.to_numeric(dfMovies.year, errors='coerce').astype('Int64')
dfMovies.budget = pd.to_numeric(dfMovies.budget, errors='coerce').astype('Int64')
dfMovies.worlwide_gross_income = pd.to_numeric(dfMovies.worlwide_gross_income, errors='coerce').astype('Int64')

In [19]:
def listEncode(df, col):
  if type(col) != list:
    col = [col]
  
  toEncode = df[col[0]]
  for element in range(1, len(col)):
    toEncode += ", " + df[col[element]]
  one_hot = toEncode.str.get_dummies(sep=', ')
  df = df.drop(col,axis = 1)
  df = df.join(one_hot)
  return df

In [45]:

def enCodeTopCategories(df, col, num_to_encode): # one hot encodes the top "num_to_encode" values present in the df column "col"
    new_df = df.copy() # copy to avoid cascading changes
    cols_to_encode = new_df[col].value_counts()[:num_to_encode].index.tolist()
    # print(new_df[col].value_counts()[:num_to_encode]) # uncomment this line to see value counts of the top num_to_encode values
    
    for category in cols_to_encode: # one hot encode
        new_df[str(col) + "_" + str (category)] = np.where(new_df[col] == category, 1, 0) # replace each of the top N categories with 1, if not a top N category, replace with 0
    new_df = new_df.drop(col,  axis=1) #remove old column
    return new_df


In [49]:
# moviesToEncode = ["genre", "country", "language"] 
# dfMovies = listEncode(dfMovies, moviesToEncode)

# we will likely need to encode the entire "years" column, shouldnt need to limit to top 100 years since it only spans over ~100 integer years  
X = dfMovies
print(X.shape) 
# right now, these categorical columns still have multiple names such as "Stephen King, Christopher Nolan"
X = enCodeTopCategories(X, 'director', 100) # one hot encode top 100 directors only
X = enCodeTopCategories(X, 'writer', 100) # one hot encode top 100 writers only
X = enCodeTopCategories(X, 'country', 100) # one hot encode top 100 directors only
X = enCodeTopCategories(X, 'language', 50)
print(X.shape)


dfMovies.to_csv('test.csv', index=False)




(85855, 12)


KeyError: 'langauge'

In [22]:
X = dfMovies
print(X)



      imdb_title_id                           title  year  \
0         tt0000009                      Miss Jerry  1894   
1         tt0000574     The Story of the Kelly Gang  1906   
2         tt0001892                  Den sorte drøm  1911   
3         tt0002101                       Cleopatra  1912   
4         tt0002130                       L'Inferno  1911   
...             ...                             ...   ...   
85850     tt9908390                         Le lion  2020   
85851     tt9911196  De Beentjes van Sint-Hildegard  2020   
85852     tt9911774       Padmavyuhathile Abhimanyu  2019   
85853     tt9914286               Sokagin Çocuklari  2019   
85854     tt9914942      La vida sense la Sara Amat  2019   

                           genre  duration           country       language  \
0                        Romance        45               USA           None   
1        Biography, Crime, Drama        70         Australia           None   
2                          Dra